In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from tensorflow import keras

In [ ]:
odata = pd.read_csv("FULLUWI.csv")
data = odata.copy()
SR = keras.models.load_model("stu_regmodel.h5")
FC = keras.models.load_model("fac_model.h5")
RC = keras.models.load_model("room_model.h5")
LRC = keras.models.load_model("lim_roommodel.h5")
LC = keras.models.load_model("lec_model.h5")

In [ ]:
#input
entry ={
     "Subject" : ['ACCT','COMP'],
     "CourseCode":[1005,1012],
     "CourseType": ['M','M'],
     "CourseType1": ['M11','M11'],
     "Semester": [1,1],
     "Year": ['2022/2023','2022/2023'],
     "Level": ['1','1'],
     "Faculty": ['FSS','FST'],
     "Location": ['Mona','Mona'],
     "Lecturer": ['Thomas-Stone, Joan', 'Beckford, Carl'],
     "Lecturer2": ['No','No'],
     "Room": ['ONLINE','ONLINE'],
     "Students": [0,150],
     "Seats": [160,160]
}

In [ ]:
connect = {"S":SR,
           "F":FC,
           "R":RC,
           "LR":LRC,
           "L":LC}

In [ ]:
features={
        "Subject": "string_categorical",
        "CourseCode": "integer_categorical",
        "CourseType": "string_categorical",
        "CourseType1": "string_categorical",
        "Semester": "integer_categorical",
        "Year": "string_categorical",
        "Faculty": "string_categorical",
        "Level": "string_categorical",
        "Location": "string_categorical",
        "Lecturer": "string_categorical",
        "Lecturer2": "string_categorical",
        "Room": "string_categorical",
        "Students": "float_normalized",
        "Seats": "float_normalized"
    }

def extraLec(name):
  if len(name.split())>3:
    return 'Yes'
  else:
    return 'No'

In [ ]:
attr = "S"
cat = True
lim = False
match attr:
  case "S":
    target = "Students"
    cat = False
    column = ['Attribute']
    data['Lecturer2'] = odata.apply(lambda x: extraLec(x.Lecturer), axis = 1)
    data['Lecturer']  = odata.apply(lambda x: ' '.join(x.Lecturer.split(' (Primary)')[0].split()[-2:]), axis=1)
    features.pop(target)
  case "F":
    target = "Faculty"
    column = ['Attribute','Students','Seats']
    features.pop('Students')
    features.pop('Seats')
    features.pop('Lecturer2')
    features.pop(target)
  case "R":
    target = "Room"
    column = ['Attribute']
    data['Lecturer2'] = odata.apply(lambda x: extraLec(x.Lecturer), axis = 1)
    data['Lecturer']  = odata.apply(lambda x: ' '.join(x.Lecturer.split(' (Primary)')[0].split()[-2:]), axis=1)
    features.pop(target)
  case "LR":
    target = "Room"
    cat = False
    lim = True
    column = ['Attribute']
    data['Room'] = odata.apply(lambda x: x.Room=='ONLINE',axis =1)
    features.pop(target)
    features.pop('Lecturer2')
  case "L":
    target = "Lecturer"
    column = ['Attribute']
    data['Lecturer']  = odata.apply(lambda x: ' '.join(x.Lecturer.split(' (Primary)')[0].split()[-2:]), axis=1)
    features.pop(target)
    features.pop('Lecturer2')

In [ ]:
if not cat:
 dataframe = data.loc[:,~data.columns.isin(column)].copy()
 labels = dataframe.pop(target)
 feature_space = tf.keras.utils.FeatureSpace(
    features = features,
    output_mode="concat",
 )
 dataset = tf.data.Dataset.from_tensor_slices(dict(dataframe))

 feature_space.adapt(dataset)

 output_vector = feature_space(dict(dataframe))
else:
  dataframe = data.loc[:,~data.columns.isin(column)].copy()
  dataframe[target] = pd.Categorical(data[target]).codes
  labels = dataframe.pop(target)
  feature_space = tf.keras.utils.FeatureSpace(
    features=features,
    output_mode="concat",
  )
  dataset = tf.data.Dataset.from_tensor_slices(dict(dataframe))

  feature_space.adapt(dataset)

  output_vector = feature_space(dict(dataframe))

In [ ]:
#predict
if lim:
   print((connect[attr].predict(feature_space(entry)) > 0.5).astype(int))
elif cat:
   print(np.argmax(connect[attr].predict(feature_space(entry)),axis=1))
else:
   print(connect[attr].predict(feature_space(entry)))